# Predicting Stock Prices

We will work with data from the [S&P500 Index](https://en.wikipedia.org/wiki/S%26P_500). 


In [66]:
import pandas as pd
from datetime import datetime
data = pd.read_csv('sphist.csv')
data['Date'] = pd.to_datetime(data['Date'])

data.sort_values('Date', ascending=True, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16590 entries, 16589 to 0
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       16590 non-null  datetime64[ns]
 1   Open       16590 non-null  float64       
 2   High       16590 non-null  float64       
 3   Low        16590 non-null  float64       
 4   Close      16590 non-null  float64       
 5   Volume     16590 non-null  float64       
 6   Adj Close  16590 non-null  float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 1.0 MB


In [67]:
#Computing rolling average price for previous 5 days
data['Day_5'] = data['Close'].rolling(window=5).mean()
#Computing rolling average price for previous 30 days
data['Day_30'] = data['Close'].rolling(window=30).mean()
#Computing rolling average price for previous 365 days
data['Day_365'] = data['Close'].rolling(window=365).mean()
#Computing rolling average volume for previous 5 days
data['Vol_5'] = data['Volume'].rolling(window=5).mean()
#Computing rolling average volume for previous 365 days
data['Vol_365'] = data['Volume'].rolling(window=365).mean()

#Shifting all the new columns 1 day forward
data[['Day_5', 'Day_30', 'Day_365', 'Vol_5', 'Vol_365']] = data[['Day_5', 'Day_30', 'Day_365', 'Vol_5', 'Vol_365']].shift(1)

data.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close,Day_5,Day_30,Day_365,Vol_5,Vol_365
16589,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,NaN,NaN,NaN,NaN,NaN
16588,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,NaN,NaN,NaN,NaN,NaN
16587,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,NaN,NaN,NaN,NaN,NaN
16586,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,NaN,NaN,NaN,NaN,NaN
16585,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,NaN,NaN,NaN,NaN,NaN
16584,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.900,NaN,NaN,2046000.0,NaN
16583,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000,16.974,NaN,NaN,2226000.0,NaN
16582,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000.0,16.760000,17.022,NaN,NaN,2374000.0,NaN
16581,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000.0,16.670000,16.988,NaN,NaN,2458000.0,NaN
16580,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000.0,16.719999,16.926,NaN,NaN,2722000.0,NaN


As one of our indicators needs data for the previous year, we will not have enough data to compute it for the whole first year of our historical data. We will just remove the first year from the dataset. 

In [68]:
data = data[data["Date"] > datetime(year=1951, month=1, day=2)]
data.dropna(axis=0, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16225 entries, 16224 to 0
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       16225 non-null  datetime64[ns]
 1   Open       16225 non-null  float64       
 2   High       16225 non-null  float64       
 3   Low        16225 non-null  float64       
 4   Close      16225 non-null  float64       
 5   Volume     16225 non-null  float64       
 6   Adj Close  16225 non-null  float64       
 7   Day_5      16225 non-null  float64       
 8   Day_30     16225 non-null  float64       
 9   Day_365    16225 non-null  float64       
 10  Vol_5      16225 non-null  float64       
 11  Vol_365    16225 non-null  float64       
dtypes: datetime64[ns](1), float64(11)
memory usage: 1.6 MB


We will train our algorithm on the data before 1st of January 2013 and then test it on the data for days after that date. We will use the indicators we've just created a features and remove all the original columns, except for "Close" that is our target column.

In [69]:
train = data[data['Date']<datetime(year=2013, month=1, day=1)]
test = data[data['Date']>datetime(year=2013, month=1, day=1)]

In [70]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

features = ['Day_5', 'Day_30', 'Day_365', 'Vol_5', 'Vol_365']
target = 'Close'
lr = LinearRegression()
lr.fit(train[features], train[target])
predictions = lr.predict(test[features])
mae = mean_absolute_error(test[target], predictions)
mse = mean_squared_error(test[target], predictions)
print(mae)
print(mse)

16.131114558722086
494.3732210903416


Ways to improve prediction

- Accuracy would improve greatly by making predictions only one day ahead. For example, train a model using data from 1951-01-03 to 2013-01-02, make predictions for 2013-01-03, and then train another model using data from 1951-01-03 to 2013-01-03, make predictions for 2013-01-04, and so on. This more closely simulates what you'd do if you were trading using the algorithm.
- You can also make the system real-time by writing an automated script to download the latest data when the market closes and make predictions for the next day.
- Add other indicators such as: The year component of the date, The standard deviation of the average volume over the past year, The day of week etc.